In [1]:
from os.path import join

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path = '../../raw_data/dataset_Kids/'

In [3]:
df_annotations_raw = pd.read_csv(join(path, 'annotations_raw.csv'))

In [4]:
df_annotations_raw.head(2)

,ic_id,user_hash,comment,flag_brain,flag_alpha,flag_mu,flag_eyes,flag_eyes_h,flag_eyes_v,flag_muscles,flag_heart,flag_line_noise,flag_ch_noise
0,IC000262,14bbef115b29d959063c0bd6a22a9fb011af9f3b8bd932...,NaN,False,False,False,True,True,True,False,False,False,False
1,IC000262,8fcf3741c3c1f9d881656f95fdb14715554f924081a68b...,NaN,False,False,False,False,False,True,False,False,False,False


In [5]:
df_annotations_raw['user_hash'] = df_annotations_raw['user_hash'].apply(lambda x: x[:8])

Будем учитывать только разметку экспертов, которые разметили практически все компоненты

In [6]:
df_annotations_raw['user_hash'].value_counts()

8fcf3741    618
14bbef11    613
aa020466     62
27d1f3c0     11
9630c152      3
35326260      1
33326b51      1
9f86d081      1
Name: user_hash, dtype: int64

In [7]:
users_to_use = ['8fcf3741', 
                '14bbef11']


In [8]:
df_annotations_raw = df_annotations_raw[df_annotations_raw['user_hash'].isin(users_to_use)]

Всего 620 ICs

In [9]:
len(df_annotations_raw['ic_id'].unique())

620

Оставляем только компоненты, которые размечены двумя экспертами

In [10]:
counts = df_annotations_raw['ic_id'].value_counts().to_frame()

In [11]:
counts = counts[counts['ic_id'] == 2]

In [12]:
len(counts)

611

In [13]:
df_annotations_raw = df_annotations_raw[df_annotations_raw['ic_id'].isin(counts.index)]

Выделим колонки с флагами

In [15]:
df_annotations_raw.columns

Index(['ic_id', 'user_hash', 'comment', 'flag_brain', 'flag_alpha', 'flag_mu',
       'flag_eyes', 'flag_eyes_h', 'flag_eyes_v', 'flag_muscles', 'flag_heart',
       'flag_line_noise', 'flag_ch_noise'],
      dtype='object')

In [16]:
flags = ['flag_brain', 'flag_alpha', 'flag_mu',
         'flag_eyes', 'flag_eyes_h', 'flag_eyes_v', 'flag_muscles', 'flag_heart',
         'flag_line_noise', 'flag_ch_noise']

In [17]:
df_annotations_raw[flags] = df_annotations_raw[flags].astype(int)

Присвоим флаг глаз, если есть разметка flag_eyes_h или flag_eyes_v

In [18]:
df_annotations_raw['flag_eyes'] = df_annotations_raw.apply(lambda row: max(row['flag_eyes'], row['flag_eyes_h'], row['flag_eyes_v']), axis=1)

Посчитаем метрики согласованности

In [53]:
unstacked = df_annotations_raw[['ic_id', 'user_hash', *flags]].set_index(['ic_id', 'user_hash']).unstack()
unstacked.head()

flag_brain          flag_alpha           flag_mu          flag_eyes  \
user_hash   14bbef11 8fcf3741   14bbef11 8fcf3741 14bbef11 8fcf3741  14bbef11   
ic_id                                                                           
IC000262           0        0          0        0        0        0         1   
IC000263           1        1          0        0        0        0         1   
IC000264           1        1          0        0        0        0         1   
IC000265           1        1          0        0        0        0         1   
IC000266           1        1          0        0        0        0         0   

                   flag_eyes_h          flag_eyes_v          flag_muscles  \
user_hash 8fcf3741    14bbef11 8fcf3741    14bbef11 8fcf3741     14bbef11   
ic_id                                                                       
IC000262         1           1        0           1        1            0   
IC000263         0           0        0           0        0            1   
IC000264         0           0        0           0        0            0   
IC000265         0           1        0           0        0            1   
IC000266         0           0        0           0        0            1   

                   flag_heart          flag_line_noise          flag_ch_noise  \
user_hash 8fcf3741   14bbef11 8fcf3741        14bbef11 8fcf3741      14bbef11   
ic_id                                                                           
IC000262         0          0        0               0        0             0   
IC000263         0          0        0               0        0             1   
IC000264         0          1        0               0        0             1   
IC000265         0          0        0               0        0             1   
IC000266         0          1        0               0        0             0   

                    
user_hash 8fcf3741  
ic_id               
IC000262         0  
IC000263         0  
IC000264         0  
IC000265         0  
IC000266         0

Считаем Cohen's kappa самостоятельно

In [40]:
mean_user_0 = unstacked.loc[:, ('flag_brain', users_to_use[0])].mean()
mean_user_1 = unstacked.loc[:, ('flag_brain', users_to_use[1])].mean()

In [41]:
p_chance = mean_user_0 * mean_user_1 + (1 - mean_user_0) * (1 - mean_user_1)
p_chance

0.6033627896635871

In [42]:
p_0 = (unstacked.loc[:, ('flag_brain', users_to_use[0])] == unstacked.loc[:, ('flag_brain', users_to_use[1])]).mean()

In [43]:
p_0

0.7692307692307693

In [44]:
(p_0 - p_chance) / (1 - p_chance)

0.4181856246581079

Проверяем метрику в sklearn

In [45]:
from sklearn.metrics import cohen_kappa_score

In [46]:
cohen_kappa_score(unstacked.loc[:, ('flag_brain', users_to_use[0])], unstacked.loc[:, ('flag_brain', users_to_use[1])])

0.4181856246581078

Считаем Cohen's kappa для всех флагов

In [47]:
for flag in flags:
    print('{}: kappa = {:.2f}'.format(flag, cohen_kappa_score(unstacked.loc[:, (flag, users_to_use[0])], unstacked.loc[:, (flag, users_to_use[1])])))

flag_brain: kappa = 0.42
flag_alpha: kappa = 0.13
flag_mu: kappa = 0.22
flag_eyes: kappa = 0.10
flag_eyes_h: kappa = 0.22
flag_eyes_v: kappa = 0.31
flag_muscles: kappa = 0.36
flag_heart: kappa = 0.04
flag_line_noise: kappa = -0.01
flag_ch_noise: kappa = 0.12


Смотрим разметку по всем флагам

In [48]:
scores = df_annotations_raw.groupby('ic_id')[flags].mean().reset_index()

In [49]:
counts = df_annotations_raw.groupby('ic_id')['flag_brain'].count().reset_index()
counts.columns = ['ic_id', 'n_annotations']

In [50]:
df_annotations = scores.merge(counts, on='ic_id')

In [52]:
for flag in flags:
    print(flag)
    print(df_annotations[flag].value_counts().sort_index())
    print('')

flag_brain
0.0     93
0.5    141
1.0    377
Name: flag_brain, dtype: int64

flag_alpha
0.0    538
0.5     66
1.0      7
Name: flag_alpha, dtype: int64

flag_mu
0.0    515
0.5     82
1.0     14
Name: flag_mu, dtype: int64

flag_eyes
0.0    216
0.5    332
1.0     63
Name: flag_eyes, dtype: int64

flag_eyes_h
0.0    527
0.5     72
1.0     12
Name: flag_eyes_h, dtype: int64

flag_eyes_v
0.0    555
0.5     45
1.0     11
Name: flag_eyes_v, dtype: int64

flag_muscles
0.0    299
0.5    191
1.0    121
Name: flag_muscles, dtype: int64

flag_heart
0.0    360
0.5    242
1.0      9
Name: flag_heart, dtype: int64

flag_line_noise
0.0    599
0.5     12
Name: flag_line_noise, dtype: int64

flag_ch_noise
0.0    373
0.5    200
1.0     38
Name: flag_ch_noise, dtype: int64

